## Include

In [ ]:
import os
import numpy as np
import torch
import onnx
import tvm
from tvm import relax
from tvm.relax.frontend.onnx import from_onnx

## Module

In [ ]:
onnx_model = onnx.load("vgg16_torch.onnx")
input_name = "input_2"
input_shape = (1, 3, 32, 32)
shape_dict = {input_name: input_shape}

In [ ]:
print("Inputs:")
for inp in onnx_model.graph.input:
    print(" -", inp.name)

print("Outputs:")
for out in onnx_model.graph.output:
    print(" -", out.name)

### Get module and params

In [ ]:
# Chuyển sang Relax IR
mod = from_onnx(
    onnx_model,
    shape_dict,
    keep_params_in_input=False  # True nếu muốn weight nằm ở input
)
mod.show()

In [ ]:
mod.show()

In [ ]:
mod: tvm.IRModule = relax.transform.LegalizeOps()(mod)
mod.show()

In [ ]:
mod: tvm.IRModule = relax.get_pipeline("zero")(mod)
mod.show()

## Architecture

### CPU test

In [ ]:
target = tvm.target.Target("llvm -mtriple=arm-none-linux-gnueabihf")
executable = relax.build(mod, target, exec_mode="compiled")
executable.export_library(file_name="libtvm_model.so",
                          cc="/root/Programming/linux/customs/test/arm-gnu-toolchain/bin/arm-none-linux-gnueabihf-g++",
                          options=["-nostdlib", "-shared", "-fPIC"])

### Raspberry Pi

In [ ]:
target = tvm.target.Target("llvm -mtriple=aarch64-linux-gnu")
executable = relax.build(mod, target, exec_mode="compiled")
executable.export_library(file_name="libtvm_model.so",
                          cc="/root/Programming/linux/customs/aarch64-linux-gnu/bin/aarch64-linux-gnu-g++",
                          options=["-nostdlib", "-shared", "-fPIC"])

### Native

In [ ]:
target = tvm.target.Target("llvm")
executable = relax.build(mod, target, exec_mode="compiled")
executable.export_library(file_name="libtvm_model.so",
                          options=["-nostdlib", "-shared", "-fPIC"])

### Luckfox Pico

In [ ]:
target = tvm.target.Target("llvm -mtriple=armv7-linux-gnueabihf")
executable = relax.build(mod, target, exec_mode="compiled")
executable.export_library(file_name="libtvm_model.so",
                          cc="/home/duyloc/Programming/linux/luckfox-pico/tools/linux/toolchain/arm-rockchip830-linux-uclibcgnueabihf/bin/arm-rockchip830-linux-uclibcgnueabihf-g++",
                          options=["-nostdlib", "-shared", "-fPIC"])

### Get Data

In [ ]:
# from PIL import Image
# # Input data
# img = Image.open("DefectHole/Resin.bmp").convert("RGB")
# img = img.resize((96, 96))
# img_np  = np.array(img).astype("float32")

In [ ]:
from PIL import Image
# Input data
img = Image.open("images/car.jpg").convert("RGB")
img = img.resize((32, 32))
img_np  = np.array(img).astype("float32")
img_np = img_np/255

In [ ]:
# Chuyển sang dạng NCHW (batch=1)
img_np = np.transpose(img_np, (2, 0, 1))  # (C, H, W)
cpu_data = np.expand_dims(img_np, 0)        # (1, C, H, W)

In [ ]:
if not os.path.exists("data"):
    os.makedirs("data")

In [ ]:
cpu_data.tofile("data/car.bin")

## Run native

In [ ]:
dev = tvm.cpu()

### Load model and weights

In [ ]:
# executable: tvm.runtime.Module = tvm.runtime.load_module("libtvm_model.so")

In [ ]:
# cpu_data = np.fromfile('your_structured_binary_file.bin', dtype='f4')

### Read data from file txt

In [ ]:
# import numpy as np

# # Đọc dữ liệu từ file
# with open("../../researches/Unprocess.txt", "r") as f:
#     data = f.read().strip().split()

# # Chuyển thành mảng numpy kiểu int
# arr = np.array(data, dtype=np.float32)

# # Reshape về (1, 96, 96, 3)
# cpu_data = arr.reshape((1, 96, 96, 3))

# print(cpu_data.shape)   # (1, 96, 96, 3)

In [ ]:
cpu_data = tvm.nd.array(cpu_data, dev)

In [ ]:
cpu_data

In [ ]:
vm = relax.VirtualMachine(executable, dev)
cpu_out = vm["main"](cpu_data)

In [ ]:
cpu_out

In [ ]:
with open("imagenet_classes.txt") as f:
    labels = [line.strip() for line in f]

In [ ]:
scores = cpu_out.numpy().reshape(-1)
# Lấy class có xác suất cao nhất
pred_class = int(np.argmax(scores))
confidence = float(scores[pred_class])

print(f"Predicted index: {pred_class}")
print(f"Predicted label: {labels[pred_class]}")
print(f"Confidence: {confidence:.4f}")